# Instructions: 

- แอป "ที่ปรึกษาภาษี"

1. เบื้องต้นคือต้องดูดข้อมูลจากเว็บสรรพากร ในส่วนข้อหารือภาษี

https://www.rd.go.th/66091.html

แล้วจัดเป็นชุดข้อมูล CSV แยกคำถาม, คำตอบ, เอกสารที่เกี่ยวข้อง, หัวข้อ ฯลฯ

# Scrape the data off the web pages

1. Get the information off of the website - sort it into a .jsonl file(better than .csv)
- Use a Python web scraper to achieve this. 
    - Web scraper tutorial used: https://www.geeksforgeeks.org/python-web-scraping-tutorial/
    - Use the requests module/package to get the website

Assemble the code to scrape for the all the links of the pages
- Use the bar on the left side to get all the links to the pages

In [2]:
import urllib.request
from bs4 import BeautifulSoup
from time import sleep

# Make a global variable weburl in case we want to use it elsewhere
global weburl
weburl = "https://www.rd.go.th/"

def removeSpecialChar(text):
    return text.replace(" &nbsp;", "").replace("\xa0", "").replace(":", "").replace("\t", "").replace("\r", "").strip()


# Scrape off the directory on the left side of the page to get each page off of the website
# Url can be any URL on the tax page
# Get year
def directoryScrape(url):
    # Get the website data and decode
    web = urllib.request.urlopen(url)
    data = web.read()
    decoded_data = data.decode("utf-8")

    # Use beautiful soup to analyze/scrape
    soup = BeautifulSoup(decoded_data, "html.parser")   

    # Get the table element that contains the data we want - get the children into a separate variable
    div = soup.find("div", {"class": "list-menu xs-toggle-content"})

    # Last one is not a link to the tax data
    yearLinks = [weburl + list(child.children)[0]["href"].replace("http://www.rd.go.th/", "") for child in div.contents[1].children][:-1]
    return yearLinks

# Get month
def subDirectory(yearLink):
    # Get the website data and decode
    web = urllib.request.urlopen(yearLink)
    data = web.read()
    decoded_data = data.decode("utf-8")

    # Use beautiful soup to analyze/scrape
    soup = BeautifulSoup(decoded_data, "html.parser")

    # Month table
    div = soup.find("div", {"class": "submenu"})
    # monthsList = [list(element.children)[0]["href"] for element in child.contents for child in div.children]
    monthsList = []
    for child in div.children:
        for element in child.contents:
            monthsList.append(weburl + list(element.children)[0]["href"].replace("http://www.rd.go.th/", ""))
            sleep(1)
    
    return monthsList

# Get article
def articleDirectory(monthLink):
    # Get the website data and decode
    web = urllib.request.urlopen(monthLink)
    data = web.read()
    decoded_data = data.decode("utf-8")

    # Use beautiful soup to analyze/scrape
    soup = BeautifulSoup(decoded_data, "html.parser")

    # Get the articles table - skip to tbody
    # Some articles can't find tbody forsome reason so instead of that find them by using the a attributes of them
    try:
        tbody = soup.find("tbody")
        articles = [weburl + child.contents[1].contents[1]["href"].replace("http://www.rd.go.th/", "") for child in list(tbody.contents)[1::2]]
    except:
        links = soup.find_all("a", {"target":"_self", "class":"style9"})
        # Fix issue where some sites contain just the ./ link while some contain the entire link
        articles = [weburl + link["href"].replace("http://www.rd.go.th/", "") for link in links if not "".join(link.contents[0]).isnumeric()]
    
    return articles



# Will scrape the article fed in and return the data fed in as a list
def scrapePage(page):
    # Get the website data and decode
    web = urllib.request.urlopen(page)
    data = web.read()
    decoded_data = data.decode("utf-8")

    # Use beautiful soup to analyze/scrape
    soup = BeautifulSoup(decoded_data, "html.parser")   

    # Get the table element that contains the data we want - get the children into a separate variable
    taxInfoTable = soup.find("table")
    # Solving problem with different formatting - later articles format with tbody as the container, some older don't contain tbody
    tbody = taxInfoTable.tbody
    if taxInfoTable.tbody == None:
        tbody = taxInfoTable

    # Put content/string of the children all into a list
    # Clean the data too
    data = []
    for tr in tbody:
        tr = tr.contents
        # Removing any line breaks
        tr = [tr[i] for i in range(len(tr)) if tr[i].name != "br"] 
        # If the length is > 1, that means that tr1 contains a string that is split into multiple elements but should be one
        # Combine these strings to get the appropriate function of the text
        if len(tr) > 1 and type(tr[0]) == str:
            paragraph = ""
            for text in tr:
                paragraph += text.get_text()
            data.append(removeSpecialChar(paragraph))
        else:
            for child in tr:
                # If it's a comment it will appear as a string and we don't want comments in the dataset - comments will appear as a 1 character string
                if type(child) != str:
                    # Some times the text comes in separate elements
                    # If not then that means it's a table so it goes into the exception
                    try:
                        data.append(removeSpecialChar(child.get_text()))
                    except AttributeError:
                        # Format the table into a format that can be exported to json properly and can be used to train a model
                        part = []
                        for element in child:
                            if element.name == None:
                                part.append(removeSpecialChar(element))
                            elif element.name == "table":
                                # The first TR will be the tags so put the tags into a variable
                                tags = [category.get_text() for category in element.tbody.contents[0]]
                                table = []
                                # Loop through elements 1 to 4 in the list and get the text for them
                                for i in range(1, len(element.tbody)):
                                    tr = element.tbody.contents[i]
                                    # Don't need the summary of it inside the dataset since that value can simply be calculated with the data
                                    if tr.contents[0].get_text() != "รวม":
                                        # Build the dictionary that contains the tag as the key and the contents is the tr element of the table inside it
                                        content = {tag:removeSpecialChar(tr.contents[tags.index(tag)].get_text()) for tag in tags}
                                        table.append(content)
                                part.append(table)
                        # Append it to the list
                        data.append(part)

    # Format that list into a dictionary with value and key
    dataDict = {}
    for i in range(0, len(data)-1, 2):
        # Don't add it if it has the "เลขตู้" tag since the newer ones don't have it 
        if data[i] != "เลขตู้": 
            dataDict[data[i]] = data[i+1]
    
    # Scrapped idea for format the ข้อกฎหมาย into a list - does not work consistently
    """
    # Properly format the ข้อกฎหมาย into a list if it contains the numeric characters
    citations = dataDict["ข้อกฎหมาย"]
    onlyChar = True
    for char in citations:
        if char.isnumeric():
            onlyChar = False
    if not onlyChar:
        # If the ascii is greater than 57, it is neither an integer nor a "(", ")", or "/" so therefore it is a Thai character and we can remove it
        citations = [part for part in "".join([citations[i] for i in range(len(citations)) if (not ord(citations[i]) > 57)]).strip().split(" ") if len(part) != 0]
        print(citations)
        toRemove = [citations[i-1]+citations[i] for i in range(len(citations)) if "(" in citations[i] or "(" in citations[i] and citations[i + 1] == ")"]
        print(toRemove)
        dataDict["ข้อกฎหมาย"] = [citation for citation in citations if citation not in toRemove]
    """
    # Return the dictionary
    return(dataDict)


test = scrapePage("https://www.rd.go.th//66911.html")
print(len(test.keys()))

for part in test.keys():
    print(f"{part}:\n{test[part]}\n\n\n")

# URL here can be any page that on the tax that has the directory on the left side fo the years and month

6
เลขที่หนังสือ:
กค 0702/2491



วันที่:
3 พฤษภาคม 2567



เรื่อง:
ภาษีธุรกิจเฉพาะ กรณีรายได้จากดอกเบี้ยรับจากการซื้อตั๋วแลกเงิน



ข้อกฎหมาย:
มาตรา 91/2 (5) มาตรา 91/5 (5) และมาตรา 91/12 แห่งประมวลรัษฎากร



ข้อหารือ:
นาง ก. ประกอบธุรกิจลงทุนในตราสารหนี้ ได้ทำสัญญาจัดการกองทุนส่วนบุคคลแต่งตั้งให้บริษัทหลักทรัพย์จัดการกองทุน (บลจ.) เป็นผู้รับมอบอำนาจและเป็นผู้บริหารกองทุนส่วนบุคคล “นาง ก. โดย บลจ.” ซึ่ง บลจ. ได้ลงทุนด้วยการซื้อตั๋วแลกเงินจากบริษัทผู้ออกตั๋วและระบุชื่อ “นาง ก. โดย บลจ.” เป็นผู้ทรงตั๋วเงิน ทั้งนี้ บลจ. จะคิดค่าธรรมเนียมในการบริหารจากนาง ก. โดยเมื่อตั๋วแลกเงินครบกำหนดชำระหนี้หรือถึงกำหนดชำระเมื่อทวงถาม บริษัทผู้ออกตั๋วจะจ่ายเงินตามมูลค่าตั๋วแลกเงินพร้อมดอกเบี้ยตามอัตราที่กำหนดไว้ให้กับผู้ซื้อตั๋วแลกเงิน และได้หักภาษีเงินได้ ณ ที่จ่าย สำหรับดอกเบี้ยที่จ่ายให้กับกองทุนส่วนบุคคล “นาง ก. โดย บลจ.” ในอัตราร้อยละ 15.0 นาง ก. จึงหารือว่า ดอกเบี้ยที่ได้รับจากการลงทุนในตั๋วแลกเงินดังกล่าวต้องเสียภาษีธุรกิจเฉพาะหรือไม่



แนววินิจฉัย:
กรณีตามข้อเท็จจริงที่หารือมานั้น นาง ก. ได้จัดต

Run all the functions we just wrote above

In [3]:
from tqdm import tqdm
import json

# Declare all the variables needed
yearPages = directoryScrape("https://www.rd.go.th/66091.html")
# Adjust yearPages to compensate for any years already done:
yearPages = yearPages[22:]

totalData = {}
# check to see if any of the articles we are about to process is already inside the data
with open("./taxdata.jsonl", "r", encoding="UTF-8") as file:
    try:
        totalData = json.load(file)
    except:
        pass


# Start the scraping loop from where we last left off
for year in tqdm(yearPages):
    print(f"Year: {year}")
    monthsList = subDirectory(year)
    for month in monthsList:
        print(f"Month: {month}")
        articleList = articleDirectory(month)
        # Update the article list for already used keys
        articleList = [article for article in articleList if article not in list(totalData.keys())]
        sleep(1)
        for article in articleList:
            print(f"Article: {article}")
            try:
                data = scrapePage(article)
                if len(data.keys()) == 6:
                    print(data)
                    totalData[article] = data
                    # Add the new data to the end
                    with open("./taxdata.jsonl", "w", encoding="UTF-8") as file:
                        json.dump(totalData, file, ensure_ascii=False)
                else:
                    print("Article format invalid - will not add")
            except Exception as e:
                print(f"Article unable to be scraped - {e}")
            finally:
                sleep(2)






  0%|          | 0/6 [00:00<?, ?it/s]

Year: https://www.rd.go.th//24850.html
Month: https://www.rd.go.th//24850.html
Month: https://www.rd.go.th//24851.html
Month: https://www.rd.go.th//24852.html
Month: https://www.rd.go.th//24843.html
Month: https://www.rd.go.th//24844.html
Month: https://www.rd.go.th//24845.html
Month: https://www.rd.go.th//24846.html
Month: https://www.rd.go.th//24847.html
Month: https://www.rd.go.th//24848.html
Month: https://www.rd.go.th//24849.html


 17%|█▋        | 1/6 [00:25<02:07, 25.45s/it]

Year: https://www.rd.go.th//25205.html
Month: https://www.rd.go.th//25205.html
Article: https://www.rd.go.th/25349.html
{'เลขที่หนังสือ': 'กค 0811/11620', 'วันที่': '17 ธันวาคม 2544', 'เรื่อง': 'ภาษีเงินได้หัก ณ ที่จ่าย กรณีการขนส่งสินค้าทางทะเล', 'ข้อกฎหมาย': 'คำสั่งกรมสรรพากร ที่ ท.ป.4/2528', 'ข้อหารือ': 'สมาคมผู้รับจัดการขนส่งสินค้าระหว่างประเทศ (THAI INTERNATIONAL FREIGHT FORWARDERS ASSOCIATION ) ได้หารือกรมสรรพากรเกี่ยวกับภาษีเงินได้หัก ณ ที่จ่าย กรณีค่าขนส่ง สินค้าว่า หมายถึง ค่าระวาง ค่าธรรมเนียม และประโยชน์อื่นใดที่เกี่ยวกับการขนส่งสินค้า ใช่หรือไม่ ซึ่งกรมสรรพากรได้ตอบข้อหารือสมาคมฯ ตามหนังสือที่ กค 0811/10115 ลงวันที่ 22 ตุลาคม 2544 สรุปสาระสำคัญดังนี้ 1. กรณีผู้ใช้บริการ (Shipper หรือ Forwarder) จ่ายค่าระวาง ค่าธรรมเนียมและ ประโยชน์อื่นใดตามที่ระบุในใบตราส่งสินค้า (Master Bill of Lading) ให้กับสายการเดินเรือ โดยสายการเดินเรือออกใบเสร็จรับเงินในนามผู้ใช้บริการ หรือกรณีเจ้าของสินค้า (Shipper) จ่าย ค่าระวาง ค่าธรรมเนียม และประโยชน์อื่นใดตามที่ระบุในใบตราส่งสินค้าของสายการเดินเร

 33%|███▎      | 2/6 [05:49<13:24, 201.22s/it]

...

 67%|██████▋   | 4/6 [17:16<09:57, 298.60s/it]

Year: https://www.rd.go.th//23274.html
Month: https://www.rd.go.th//23274.html
Article: https://www.rd.go.th/23275.html
{'เลขที่หนังสือ': 'กค 0811/16531', 'วันที่': '1 ธันวาคม 2541', 'เรื่อง': 'ภาษีเงินได้นิติบุคคล กรณีการคำนวณอัตราแลกเปลี่ยนเงินตราต่างประเทศ', 'ข้อกฎหมาย': 'มาตรา 65 ทวิ (5), คำสั่งกรมสรรพากร ที่ ท.ป.72/2540ฯ', 'ข้อหารือ': 'บริษัทฯ เป็นบริษัทที่ได้รับการส่งเสริมการลงทุน โดยมีรายได้จากการประกอบกิจการทั้งที่  ได้รับการส่งเสริมการลงทุน และที่ไม่ได้รับการส่งเสริมการลงทุน บริษัทฯ มีรอบระยะเวลาบัญชีเริ่ม  ตั้งแต่วันที่ 21 พฤษภาคม และสิ้นสุดวันที่ 20 พฤษภาคม ของปีถัดไปในรอบระยะเวลาบัญชี 21  พฤษภาคม 2540 - 20 พฤษภาคม 2541 บริษัทฯ มีผลขาดทุนจากการตีราคาทรัพย์สินหรือหนี้สินที่มีค่า  หรือราคาเป็นเงินตราต่างประเทศที่เหลืออยู่ในวันสุดท้ายของรอบระยะเวลาบัญชีเป็นเงินตราไทยตาม  มาตรา 65 ทวิ (5) วรรคหนึ่ง แห่งประมวลรัษฎากร ซึ่งตามคำสั่งกรมสรรพากร ที่ ท.ป.72/2540 ฯ  บริษัทฯ สามารถจะเลือกนำผลขาดทุนดังกล่าวมาคำนวณเป็นรายจ่ายของบริษัทฯ ได้ตามวิธีใดวิธีหนึ่งที่  ระบุไว้ในคำสั่งดังกล่าว เป็น

 83%|████████▎ | 5/6 [22:56<05:13, 313.50s/it]

Year: https://www.rd.go.th//22904.html
Month: https://www.rd.go.th//22904.html
Article: https://www.rd.go.th/22915.html
{'เลขที่หนังสือ': 'กค 0802/17117', 'วันที่': '18 ธันวาคม 2540', 'เรื่อง': 'ภาษีเงินได้หัก ณ ที่จ่าย และภาษีมูลค่าเพิ่ม กรณีบริษัทได้รับการส่งเสริมการลงทุน', 'ข้อกฎหมาย': 'ประเด็นปัญหา', 'ข้อหารือ': 'บริษัทฯ ได้รับการส่งเสริมการลงทุนในกิจการผลิตสายพานลำเลียง (CHAIN CONVEYOR) ประเภทการผลิตเครื่องจักร หรืออุปกรณ์สำหรับงานอุตสาหกรรม \x93ระบบสายพานลำเลียง\x94 เป็นเครื่องจักรที่มีความสำคัญในระบบการผลิตแบบอุตสาหกรรม โดยเฉพาะอย่างยิ่งอุตสาหกรรมหนัก เช่น การผลิตชิ้นส่วนรถยนต์ การประกอบรถยนต์ เป็นต้น กล่าวคือระบบสายพานลำเลียงจะช่วยผ่อนแรงในการขนชิ้นงานไปผ่านกระบวนการผลิตในแต่ละขั้นตอน และช่วยให้การผลิตมีประสิทธิภาพและประสิทธิผลยิ่งขึ้น การผลิตระบบสายพานลำเลียงจำเป็นต้องคำนึงถึงสายการผลิต (PRODUCTION LINE) ตลอดจนการจัดวางเครื่องจักรที่ใช้ในการผลิตทั้งหมดด้วย ซึ่งขั้นตอนการผลิตที่บริษัทฯ ได้ขอรับการส่งเสริมการลงทุนและได้รับอนุมัติมีดังนี้ คือ การออกแบบทางวิศวกรรม (CAD) จัดหาวัสดุ

100%|██████████| 6/6 [24:17<00:00, 243.00s/it]


# Convert the data to vectors by embedding it using Azure OpenAI Langchain
- Same as we used with the previous Search engine


In [3]:
from langchain_openai import AzureOpenAIEmbeddings
from dotenv import dotenv_values

secrets = dotenv_values("../.env")

# Initialize embeddings
embeddings = AzureOpenAIEmbeddings(deployment=secrets["OPENAI_EMBEDDINGS_DEPLOYMENT"])

# Function to embed text
def embed_text(text):
    return embeddings.embed_query(text)


In [32]:
# Load the data file
import json
dataset = {}
with open("./taxdata.jsonl", "r", encoding="UTF-8") as file:
    dataset = json.load(file)
    file.close()

# Already cleaned it out so now we don't run it
"""# Cleaned the bad ones out
for key in list(dataset.keys()):
    if "ข้อหารือ" not in dataset[key].keys():
        dataset.pop(key)
with open("./taxdata.jsonl", "w", encoding="UTF-8") as f:
    json.dump(dataset, f, ensure_ascii=False)"""



# Embed the data
embedData = {}
for key in dataset.keys():
    embedData[key] = embed_text(dataset[key]["ข้อหารือ"].replace("\n", ""))

print("Embedding finished - saving")

# Save the embedded tax data
with open("./embedtaxdata.jsonl", "w", encoding = "UTF-8") as file:
    json.dump(embedData, file, ensure_ascii=False)

print("File saved")

Embedding finished - saving


TypeError: dump() missing 1 required positional argument: 'fp'

# Data Visualization with t-SNE
- Do the same as we did for the previous Search Engine to visualize the data groupings

In [39]:
from sklearn.manifold import TSNE
import plotly.express as plot
import numpy
import json
from sklearn.decomposition import TruncatedSVD

with open("./embedtaxdata.jsonl", "r", encoding="UTF-8") as file:
    data = json.load(file)
    vectorData = numpy.array(list(data.values()))

    # Do dimensionality reduction first w/ Truncated SVD
    tsvd = TruncatedSVD(n_components=2)
    vectorData_tsvd = tsvd.fit_transform(vectorData)

    fig = plot.scatter(x = vectorData_tsvd[:, 0], y = vectorData_tsvd[:, 1])
    fig.show()

    # Initialize tsne and fit the data to tsne
    tsne = TSNE(n_components=2)
    vectorData_tsne = tsne.fit_transform(vectorData_tsvd)

    # Lower divergence means the data has more patterns that can be trained for more efficiency
    print(tsne.kl_divergence_)

    # Visualize it into a figure
    # fig = plot.scatter(x = vectorData_tsne[:, 0], y = vectorData_tsne[:, 1])
    fig = plot.scatter(x = vectorData_tsne[:, 0], y = vectorData_tsne[:, 1])
    fig.show()
    



0.5203408598899841


# Create the vector database for it using Milvus lite
- Easier to use Pymilvus in WSL Ubuntu than Windows so switch to WSL in VSCode for this

1. Open the data file in python for the embedded data
2. Create the Milvus client
3. Turn the database into a Milus collection
4. Create a simple user search function and search through the database - return the top 10 results from the Milvus database

**This code portion below will create a *new* database - only run if the data is changed since the database is already saved locally**

In [3]:
from pymilvus import MilvusClient
import json
import numpy

# Open the embedded file and turn them into values
file = open("./embedtaxdata.jsonl", "r", encoding="UTF-8")
data = json.load(file)
vectorData = numpy.array(list(data.values()))
vectorKeys = list(data.keys())
file.close()

# Create/get the database
mil = MilvusClient("taxinfo.db")
collectionName = "tax"

# If the collection already exists, drop it
if mil.has_collection(collectionName):
    mil.drop_collection(collectionName)

# Create the new collection
# Dimensions = the dimensions in one embedding for a particular sample; for azure embedding 3 small dimensions = 1536
mil.create_collection(collection_name=collectionName, dimension=vectorData.shape[1])

# Milvus database entry format: #{id:id, vector: vector, text:text} - for the ID number just use the of the value in the normal list 
# Reformat out stuff to line up with that
# Not putting in the text - id is enough to identify
milDict = [{"id": i, "vector":vectorData[i]} for i in range(len(vectorData))]
mil.insert(collectionName, milDict)



{'insert_count': 1644,
 'ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215

Load the database(if kernel reset or anything use it)

In [5]:
from pymilvus import MilvusClient

mil = MilvusClient("taxinfo.db")
collectionName = "tax"

- Input - user query
- Output - top 10 search results for that in the database

In [6]:
import json

query = input("Enter query: ")

# Embed_text function was declared way above, might need to rerun that cell in order to have it be effective here
milResults = mil.search(collection_name=collectionName, data=[embed_text(query)], limit=10)

data = {}
with open("./taxdata.jsonl", "r", encoding="UTF-8") as file:
    data = json.load(file)

# Get the results in the database for the ones given by Milvus
results = [list(data.keys())[dataDict["id"]] for dataDict in milResults[0]]

# Print it out for the user
print(f"Results for query: {query}")
for result in results:
    print(f"\n{result} - {data[result]}")

Results for query: การหลีกเลี่ยงภาษี

https://www.rd.go.th//66827.html - {'เลขที่หนังสือ': 'กค 0702/พ./013', 'วันที่': '13 ธันวาคม 2565', 'เรื่อง': 'ภาษีมูลค่าเพิ่ม กรณีการลงลายมือชื่อขอคืนภาษี', 'ข้อกฎหมาย': 'มาตรา 7 แห่งประมวลรัษฎากร', 'ข้อหารือ': 'บริษัท ก. ได้ยื่นแบบ ภ.พ.30 ระบุขอคืนภาษีมูลค่าเพิ่มเป็นเงินสด โดยกรรมการบริษัท ก. ลงลายมือชื่อเป็นลายเซ็นในช่องคำรับรองข้อความที่แสดงในแบบแสดงรายการเป็นภาษาญี่ปุ่น แต่ในช่องการขอคืนภาษี ได้ลงลายมือชื่อเป็นตัวหนังสือภาษาอังกฤษ ซึ่งแตกต่างกัน การลงลายมือชื่อดังกล่าวถือว่าเป็นการลงลายมือชื่อ ที่ชอบด้วยประมวลรัษฎากรหรือไม่', 'แนววินิจฉัย': 'มาตรา 7 แห่งประมวลรัษฎากร บัญญัติว่า “บรรดารายการ รายงาน หรือเอกสารอื่นซึ่งบริษัท หรือห้างหุ้นส่วนจดทะเบียนเป็นนิติบุคคลต้องทำยื่นนั้น ให้กรรมการหรือผู้เป็นหุ้นส่วนหรือผู้จัดการเป็นผู้ ลงลายมือชื่อ” ซึ่งบทบัญญัติดังกล่าวไม่ได้มีการกำหนดหลักเกณฑ์และวิธีการในการลงลายมือชื่อตลอดจนวิธีการพิสูจน์ลายมือชื่อของผู้เป็นเจ้าของลายมือชื่อไว้เป็นการเฉพาะ ดังนั้น หากบริษัท ก. สามารถพิสูจน์ได้ว่าการลงลายมือชื่อของกรรมกา

# Turn it into a RAG
- Utilize ChatGPT API for this

Steps:
1. Initialize ChatGPT into the program(might have to install a few packages for pip)
2. Ask user query
3. Get the top search results from the vector database we alr made
4. Pass the query and the search results into ChatGPT
    - Tell it to formulate an appropriate answer from the search results we gave for the query
5. Print the output ChatGPT returned to the User

In [8]:
# Initialize the AI model we will use(ChatGPT)
from openai import AzureOpenAI
import os
from dotenv import dotenv_values

secrets = dotenv_values("../../.env")


client = AzureOpenAI(azure_endpoint=secrets["AZURE_OPENAI_ENDPOINT"], 
            api_version="2024-02-01", 
            api_key=os.environ["AZURE_OPENAI_API_KEY"],
            azure_deployment="gpt4o")

# Use the results from the query on the previous cell and feed it into GPT
completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[{"role": "system", "content": "You are a lawyer that accepts a query from the user along with some data which contains anwers for similar queries and answers to those queries to the one that the user inputted. The queries will be in an array with each entry being in the form of query:[answer, citation]. You will use that data given and make an educated guess on the correct answer to the user's query in one paragraph. Don't tell the user that you are making an educated guess. Sound sure of your answer. "},
              {"role": "user", "content": f"User query: {query}\nSimilar queries and answers: { {data[result]['ข้อหารือ']:[data[result]['แนววินิจฉัย'], 'เลขที่หนังสือ: '+data[result]['เลขที่หนังสือ']] for result in results} }\nProvide answer to user query, along with some citations for information that you believe is relavant to the user's query and support your answer with evidence. "}]
)

print(completion.choices[0].message.content)

การหลีกเลี่ยงภาษีเป็นการกระทำที่ผิดกฎหมายและมีบทลงโทษตามประมวลรัษฎากร โดยเฉพาะมาตรา 54 แห่งประมวลรัษฎากรที่กำหนดโทษในการหลบเลี่ยงภาษีอย่างชัดเจน การกระทำทางกลโกงใด ๆ ที่ออกแบบมาเพื่อหลีกเลี่ยงภาระภาษี เช่น การปกปิดรายได้ การเสียภาษีในอัตราที่ต่ำกว่าความเป็นจริง หรือการประกอบรายการที่มิชอบล้วนเป็นการกระทำที่ไม่ชอบด้วยกฎหมาย

ฐานและวิธีการในการลงลายมือชื่อในเอกสารต่าง ๆ ซึ่งส่งผลต่อการยืนยันท่าทางทางกฎหมายของการยื่นเอกสารนั้นสามารถดูได้จาก มาตรา 7 แห่งประมวลรัษฎากรที่ระบุ "บรรดารายการ รายงาน หรือเอกสารอื่นซึ่งบริษัท หรือนิติบุคคลต้องทำยื่นนั้น ให้กรรมการหรือผู้เป็นหุ้นส่วนหรือผู้จัดการเป็นผู้ลงลายมือชื่อ" และการใช้ลายมือชื่อเป็นภาษาต่างชาติหากสามารถพิสูจน์ว่าเป็นลายมือของเจ้าของแล้วก็ถือว่าถูกต้องตามกฎหมาย (เลขที่หนังสือ: กค 0702/พ./013)

นอกจากนี้ กรณีที่มีการหลีกเลี่ยงภาษีโดยการทำธุรกรรมที่ไม่ชอบด้วยกฎหมายเช่น การไม่รับรู้รายได้หรือการหักภาษี ณ ที่จ่ายไม่ถูกต้อง จะต้องถูกประเมินและเรียกเก็บภาษีเพิ่มเติม รวมถึงการปรับและบทลงโทษตามกฎหมายด้วย ทั้งนี้ เมื่อมีการพบข้อสงสัยในการประกอบธุรกรรม